In [ ]:
#https://www.programmableweb.com/category/all/mashups?page=0

In [1]:
!pip install requests-random-user-agent
!pip install pandas
!pip install bs4
!pip install lxml

     |████████████████████████████████| 51kB 5.3MB/s 


In [2]:
import multiprocessing
multiprocessing.cpu_count()

2

In [6]:

import requests
import requests_random_user_agent
#s = requests.Session()
#print(s.headers['User-Agent'])

# Without a session
resp = requests.get('https://httpbin.org/user-agent')
print(resp.json()['user-agent'])

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36 Edg/89.0.774.50


In [7]:
import concurrent.futures
import requests
import requests_random_user_agent
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

def _download_MASH(url):

  df_tempSDK = pd.DataFrame()

  rq = requests.get(url)
  
  if rq.status_code == 404 or rq.status_code == 403: ## Handle more error codes...
    exit

  main_data = rq.text

  # dataset from table
  df_tempSDK = pd.read_html(main_data, index_col=0)[0]

  main_soup = BeautifulSoup(main_data, 'html.parser')
  
  main_names = main_soup.find_all('tr')[1:245]

  list_urlSDK = []

  # Head url for meta_url
  head_Url= 'https://www.programmableweb.com'

  for row in main_names:
       text = row.find_all('td')[0]      
       list_urlSDK.append( (head_Url + str(text).partition('<a href="')[2].partition('">')[0])) # Meta URL 


  df_tempSDK['Meta_Url'] = list_urlSDK

  return df_tempSDK


def download_MASH(bulk_urls, num_Splits):
  df_temp = pd.DataFrame()

  # Split urls
  urls_splited  = np.array_split(bulk_urls, num_Splits) # max workers

  tasks = []

  for split in range(len(urls_splited)):
    with concurrent.futures.ThreadPoolExecutor(max_workers = len(urls_splited)) as executor:
      for url in urls_splited[split]:    
        tasks.append(executor.submit(_download_MASH, url))
       ## mb call with bulk urls instead of one by one
  # Union
  for result in tasks:
    df_temp = df_temp.append(result.result())

  return df_temp


In [15]:
MASH_urls=[]
for i in range(2): ## web pages?.. 22+1
    main_url = 'https://www.programmableweb.com/category/all/mashups?page=' + str(i)
    MASH_urls.append(main_url)

In [16]:
#SDK_urls
df_MASH = pd.DataFrame()
df_MASH = download_MASH(MASH_urls, 5) # Num workers
#df_SDK.drop('Published', inplace=True, axis=1)
df_MASH

,Description,Category,Submitted,Meta_Url
Mashup Name,,,,
Salonist,Salonist software is a smart tool for handling...,Software-as-a-Service,04.01.2021,https://www.programmableweb.com/mashup/salonist
WaifuAI Desktop,Talk to cute waifu on Desktop (Windows/Mac/Lin...,Artificial Intelligence,03.30.2021,https://www.programmableweb.com/mashup/waifuai...
WaifuAI Android,Talk to cute waifu on Android. Waifus are anim...,Artificial Intelligence,03.30.2021,https://www.programmableweb.com/mashup/waifuai...
SimplyCRMApps,"Gain more customers, optimize your business an...",B2B,01.13.2021,https://www.programmableweb.com/mashup/simplyc...
Care Steps,"CareSteps – Robust, Paperless Business Solutio...",Digital Asset Management,11.21.2020,https://www.programmableweb.com/mashup/care-steps
Scaleo,Are you a business owner looking to add an aff...,Marketing,11.12.2020,https://www.programmableweb.com/mashup/scaleo
QuickMocker,QuickMocker is an online API tool that allows ...,Tools,11.09.2020,https://www.programmableweb.com/mashup/quickmo...
EspoCRM,EspoCRM is an open-source web application desi...,Customer Relationship Management,10.19.2020,https://www.programmableweb.com/mashup/espocrm-0
Email2Go,Email2Go is an online service that helps you c...,Software-as-a-Service,10.06.2020,https://www.programmableweb.com/mashup/email2go


In [17]:
# Creates new columns
df_MASH['Related APIs'] = ""
df_MASH['Categories'] = ""
df_MASH['URL'] = ""
df_MASH['Company'] = ""
df_MASH['App Type'] = ""

In [18]:
def _download_meta_url(df_source):

  for i in range(len(df_source)):

    #print(df_source)
    meta_url = df_source['Meta_Url'][i]
    rq = requests.get(meta_url)

    if rq.status_code == 404 or rq.status_code == 403: ## Handle more error codes...
      continue

    meta_data = rq.text
    meta_soup = BeautifulSoup(meta_data, 'html.parser')

    # Update Description from the meta url
    meta_description = str(meta_soup.find('div', class_='tabs-header_description')).partition('">')[2].partition('</')[0]
    
    df_source['Description'][i] = meta_description 

    meta_specs = meta_soup.find('div', class_='section specs')

    for lab in meta_specs.select("label"):   

      # Search for Languages
      if (lab.text.lower().find("related apis") > -1):
          #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Related APIs'][i] =   lab.find_next_sibling().text

      # Search for Provider
      if (lab.text.lower().find("categories") > -1):
          #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Categories'][i] =   lab.find_next_sibling().text

      # Search for URL
      if (lab.text.lower().find("url") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['URL'][i] =   lab.find_next_sibling().text

      # Search for Repo
      if (lab.text.lower().find("company") > -1):
          #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Company'][i] =   lab.find_next_sibling().text

      # Search for Provider
      if (lab.text.lower().find("app type") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['App Type'][i] =   lab.find_next_sibling().text


    #print(df_source)

  return df_source


def download_meta_url(df_sourceCode, num_Splits):
  # Split dataframe
  dtframe_splited  = np.array_split(df_sourceCode, num_Splits) # max workers
  tasks = []

  for split in range(len(dtframe_splited)):
    with concurrent.futures.ThreadPoolExecutor(max_workers = len(dtframe_splited)) as executor:
      # Download dataframes splited
      tasks.append(executor.submit(_download_meta_url, dtframe_splited[split]))

  df_temp = pd.DataFrame()
  # Union
  for result in tasks:
    df_temp = df_temp.append(result.result())

  return df_temp

In [19]:
df_MASH_metaData = pd.DataFrame()
df_MASH_metaData = download_meta_url(df_MASH, 5)

df_MASH_metaData

,Description,Category,Submitted,Meta_Url,Related APIs,Categories,URL,Company,App Type
Mashup Name,,,,,,,,,
Salonist,"<div class=""field field-name-body field-type-t...",Software-as-a-Service,04.01.2021,https://www.programmableweb.com/mashup/salonist,SaasGrid,"Software-as-a-Service, Sales, Software-as-a-Se...",https://salonist.io/,Shrivra,Web
WaifuAI Desktop,"<div class=""field field-name-body field-type-t...",Artificial Intelligence,03.30.2021,https://www.programmableweb.com/mashup/waifuai...,WaifuAI,"Artificial Intelligence, Chat, Machine Learning",https://waifuai.itch.io/waifu/,WaifuAI,Desktop
WaifuAI Android,"<div class=""field field-name-body field-type-t...",Artificial Intelligence,03.30.2021,https://www.programmableweb.com/mashup/waifuai...,WaifuAI,"Artificial Intelligence, Chat, Machine Learning",https://play.google.com/store/apps/details?id=...,WaifuAI,Mobile
SimplyCRMApps,"<div class=""field field-name-body field-type-t...",B2B,01.13.2021,https://www.programmableweb.com/mashup/simplyc...,"Outlook Notifications REST, Google Calendar, W...","B2B, Applications, Business, Contacts, Custome...",https://simply-crm.com,Simply CRM,Web
Care Steps,"<div class=""field field-name-body field-type-t...",Digital Asset Management,11.21.2020,https://www.programmableweb.com/mashup/care-steps,CSV2GEO,"Digital Asset Management, eCommerce, Office",https://care-steps.com,Scale Campaign,Mobile
Scaleo,"<div class=""field field-name-body field-type-t...",Marketing,11.12.2020,https://www.programmableweb.com/mashup/scaleo,"Shopify Billing , Shopify Admin, Shopify Checkout","Marketing, Automation",https://www.scaleo.io,,Web
QuickMocker,"<div class=""field field-name-body field-type-t...",Tools,11.09.2020,https://www.programmableweb.com/mashup/quickmo...,Mocky,"Tools, API, API Management, Application Develo...",https://quickmocker.com,,Web
EspoCRM,"<div class=""field field-name-body field-type-t...",Customer Relationship Management,10.19.2020,https://www.programmableweb.com/mashup/espocrm-0,"Google Maps, Google Calendar, Google Contacts,...","Customer Relationship Management, Business, Cu...",https://www.espocrm.com/download/,Letrium LTD,Web
Email2Go,"<div class=""field field-name-body field-type-t...",Software-as-a-Service,10.06.2020,https://www.programmableweb.com/mashup/email2go,SQLify,"Software-as-a-Service, Design, Email",https://email2go.io,Email2Go,Web


In [20]:
# Meta_Url could be used to check for updates on the source website. That uses only +600 web requests instead of +15k
# save a copy of the original dataframe to check for updates based on the meta url or other fields
df_MASH_metaData.reset_index(inplace=True)
df_MASH_metaData

,Mashup Name,Description,Category,Submitted,Meta_Url,Related APIs,Categories,URL,Company,App Type
0,Salonist,"<div class=""field field-name-body field-type-t...",Software-as-a-Service,04.01.2021,https://www.programmableweb.com/mashup/salonist,SaasGrid,"Software-as-a-Service, Sales, Software-as-a-Se...",https://salonist.io/,Shrivra,Web
1,WaifuAI Desktop,"<div class=""field field-name-body field-type-t...",Artificial Intelligence,03.30.2021,https://www.programmableweb.com/mashup/waifuai...,WaifuAI,"Artificial Intelligence, Chat, Machine Learning",https://waifuai.itch.io/waifu/,WaifuAI,Desktop
2,WaifuAI Android,"<div class=""field field-name-body field-type-t...",Artificial Intelligence,03.30.2021,https://www.programmableweb.com/mashup/waifuai...,WaifuAI,"Artificial Intelligence, Chat, Machine Learning",https://play.google.com/store/apps/details?id=...,WaifuAI,Mobile
3,SimplyCRMApps,"<div class=""field field-name-body field-type-t...",B2B,01.13.2021,https://www.programmableweb.com/mashup/simplyc...,"Outlook Notifications REST, Google Calendar, W...","B2B, Applications, Business, Contacts, Custome...",https://simply-crm.com,Simply CRM,Web
4,Care Steps,"<div class=""field field-name-body field-type-t...",Digital Asset Management,11.21.2020,https://www.programmableweb.com/mashup/care-steps,CSV2GEO,"Digital Asset Management, eCommerce, Office",https://care-steps.com,Scale Campaign,Mobile
5,Scaleo,"<div class=""field field-name-body field-type-t...",Marketing,11.12.2020,https://www.programmableweb.com/mashup/scaleo,"Shopify Billing , Shopify Admin, Shopify Checkout","Marketing, Automation",https://www.scaleo.io,,Web
6,QuickMocker,"<div class=""field field-name-body field-type-t...",Tools,11.09.2020,https://www.programmableweb.com/mashup/quickmo...,Mocky,"Tools, API, API Management, Application Develo...",https://quickmocker.com,,Web
7,EspoCRM,"<div class=""field field-name-body field-type-t...",Customer Relationship Management,10.19.2020,https://www.programmableweb.com/mashup/espocrm-0,"Google Maps, Google Calendar, Google Contacts,...","Customer Relationship Management, Business, Cu...",https://www.espocrm.com/download/,Letrium LTD,Web
8,Email2Go,"<div class=""field field-name-body field-type-t...",Software-as-a-Service,10.06.2020,https://www.programmableweb.com/mashup/email2go,SQLify,"Software-as-a-Service, Design, Email",https://email2go.io,Email2Go,Web
9,Domain Rank Checker,"<div class=""field field-name-body field-type-t...",SEO,09.29.2020,https://www.programmableweb.com/mashup/domain-...,SEMrush,"SEO, Analytics, Comparisons, Marketing, Search",https://www.seoreviewtools.com/domain-rank-che...,SEO Review Tools,Web


In [21]:
from datetime import datetime
datetime = datetime.now()

## Export the original + Meta_Url
# To CSV (index True 0,1,2...)
df_MASH_metaData.to_csv(r'/content/DataFrames/FRAME_' + datetime.now().strftime('%d_%m_%Y') + '.csv', index = True, header = True)

# To JSON (columns format index True 0,1,2...)
df_MASH_metaData.to_json(r'/content/DataFrames/FRAME_' + datetime.now().strftime('%d_%m_%Y') + '.json')